# The YUSAG FCS Elo Model
    by Matt Robinson, Yale Undergraduate Sports Analytics Group

Let's start by loading the csv of game data, sorted by date:

In [152]:
import numpy as np
import pandas as pd

In [153]:
game_df = pd.read_csv('2013-2016_NCAA_football_fcs_elo.csv')

In [154]:
game_df.head()

,year,month,day,team,opponent,location,teamscore,oppscore
0,2013,8,29,Central Ark.,Incarnate Word,1,58,7
1,2013,8,29,Chattanooga,UT Martin,1,21,31
2,2013,8,29,Dayton,Youngstown St.,-1,10,28
3,2013,8,29,Delaware,Jacksonville,1,51,35
4,2013,8,29,Eastern Ky.,Robert Morris,1,38,6


In [155]:
game_df['scorediff'] = (game_df['teamscore']-game_df['oppscore'])

In [156]:
game_df.head()

,year,month,day,team,opponent,location,teamscore,oppscore,scorediff
0,2013,8,29,Central Ark.,Incarnate Word,1,58,7,51
1,2013,8,29,Chattanooga,UT Martin,1,21,31,-10
2,2013,8,29,Dayton,Youngstown St.,-1,10,28,-18
3,2013,8,29,Delaware,Jacksonville,1,51,35,16
4,2013,8,29,Eastern Ky.,Robert Morris,1,38,6,32


We'll create an initial dictionary of elo ratings:

In [157]:
elo_ratings_dict = {}
for index, row in game_df.iterrows():
    elo_ratings_dict[row['team']] = 1500
    # also go through opponent column in case any team only in that column
    elo_ratings_dict[row['opponent']] = 1500

Now, let's go through the games and apply the elo algorithm.

In [158]:
# function to update elo ratings based on the score
def elo_update(winner_elo, loser_elo, mov):
    k = 20
    mov_multiplier = (2.2 * np.log(mov + 1))/((0.001 * (winner_elo-loser_elo))+2.2)
    
    u_winner = 1/(1+(10 ** ((winner_elo-loser_elo)/400)))
    u_loser = 1/(1+(10 ** ((loser_elo-winner_elo)/400)))
    
    s1 = 1
    s2 = 0
    
    new_winner_elo = winner_elo + k * mov_multiplier * (s1 - u_winner)
    new_loser_elo = loser_elo + k * mov_multiplier * (s2 - u_loser)
    
    return [new_winner_elo, new_loser_elo]

In [159]:
current_year = 2013
for index, row in game_df.iterrows():
    # first check if it is a new year
    year = row['year']
    if year != current_year:
        # if new year, regress all elo ratings to the mean
        for key in elo_ratings_dict:
            elo_ratings_dict[key] = (0.5*elo_ratings_dict[key]) + (0.5*1500)
        # update the year
        current_year = year
        
    # find the margin of victory
    score_diff = row['scorediff']
    mov = abs(score_diff)
        
    # check if the game was at home, away, or neutral
    if row['location'] == 1.0:
        home_team = row['team']
        away_team = row['opponent']
        
        # get corresponding elo ratings
        home_elo = elo_ratings_dict[home_team] + 65
        away_elo = elo_ratings_dict[away_team]
        
        # check if home or away team won
        if score_diff > 0: 
            new_home_elo = elo_update(home_elo,away_elo,mov)[0] - 65
            new_away_elo = elo_update(home_elo,away_elo,mov)[1]
        elif score_diff < 0:
            new_away_elo = elo_update(away_elo,home_elo,mov)[0]
            new_home_elo = elo_update(away_elo,home_elo,mov)[1] - 65 
            
        # update the ratings dict
        elo_ratings_dict[home_team] = new_home_elo
        elo_ratings_dict[away_team] = new_away_elo
        
    elif row['location'] == -1.0:
        home_team = row['opponent']
        away_team = row['team']
        
        # get corresponding elo ratings
        home_elo = elo_ratings_dict[home_team] + 65
        away_elo = elo_ratings_dict[away_team]
        
        # check if home or away team won
        if score_diff < 0: 
            new_home_elo = elo_update(home_elo,away_elo,mov)[0] - 65
            new_away_elo = elo_update(home_elo,away_elo,mov)[1]
        elif score_diff > 0:
            new_away_elo = elo_update(away_elo,home_elo,mov)[0]
            new_home_elo = elo_update(away_elo,home_elo,mov)[1] - 65 
            
        # update the ratings dict
        elo_ratings_dict[home_team] = new_home_elo
        elo_ratings_dict[away_team] = new_away_elo
            
    elif row['location'] == 0.0:
        # since neutral just arbitrarily make home and away, but don't add 65
        home_team = row['team']
        away_team = row['opponent']
        
        # get corresponding elo ratings
        home_elo = elo_ratings_dict[home_team]
        away_elo = elo_ratings_dict[away_team]
        
        # check if home or away team won
        if score_diff > 0: 
            new_home_elo = elo_update(home_elo,away_elo,mov)[0]
            new_away_elo = elo_update(home_elo,away_elo,mov)[1]
        elif score_diff < 0:
            new_away_elo = elo_update(away_elo,home_elo,mov)[0]
            new_home_elo = elo_update(away_elo,home_elo,mov)[1] 
            
        # update the ratings dict
        elo_ratings_dict[home_team] = new_home_elo
        elo_ratings_dict[away_team] = new_away_elo
    
    
    

In [160]:
elo_ratings_dict

{'Abilene Christian': 1371.974938356436,
 'Alabama A&M': 1374.3341829364481,
 'Alabama St.': 1531.3148770439902,
 'Albany (NY)': 1450.2693603080852,
 'Alcorn': 1701.8276801582988,
 'Ark.-Pine Bluff': 1161.740418109933,
 'Austin Peay': 1123.4516182364441,
 'Bethune-Cookman': 1648.4534815599309,
 'Brown': 1483.962590862064,
 'Bryant': 1507.9389540387938,
 'Bucknell': 1512.8109647936383,
 'Butler': 1419.6600888772978,
 'Cal Poly': 1566.1194861884806,
 'Campbell': 1450.2633126904798,
 'Central Ark.': 1651.5359822068558,
 'Central Conn. St.': 1248.925705135111,
 'Charleston So.': 1663.2472596991638,
 'Charlotte': 1498.2961741013578,
 'Chattanooga': 1789.5125575680909,
 'Coastal Caro.': 1664.9051818247553,
 'Colgate': 1523.1475646331128,
 'Columbia': 1248.9326673570238,
 'Cornell': 1255.826315531858,
 'Dartmouth': 1638.3716743940108,
 'Davidson': 1161.991736348733,
 'Dayton': 1764.5603427276037,
 'Delaware': 1476.5952521239183,
 'Delaware St.': 1161.103845779578,
 'Drake': 1535.9457766586243

In [161]:
# See it in sorted form
import operator
sorted_elo_rankings = sorted(elo_ratings_dict.items(), key=operator.itemgetter(1))

In [162]:
sorted_elo_rankings

[('Mississippi Val.', 1120.7456318041661),
 ('Austin Peay', 1123.4516182364441),
 ('Delaware St.', 1161.103845779578),
 ('Ark.-Pine Bluff', 1161.740418109933),
 ('Davidson', 1161.991736348733),
 ('Rhode Island', 1183.0922877079715),
 ('Valparaiso', 1216.1522264565097),
 ('Elon', 1224.3533603503838),
 ('Lafayette', 1225.7862597985782),
 ('Robert Morris', 1230.1658296357386),
 ('Central Conn. St.', 1248.925705135111),
 ('Columbia', 1248.9326673570238),
 ('Houston Baptist', 1254.9947172161642),
 ('Cornell', 1255.826315531858),
 ('Missouri St.', 1271.3286128919408),
 ('Stetson', 1282.2444050780637),
 ('VMI', 1286.0494729874811),
 ('Savannah St.', 1303.8456160769413),
 ('Georgetown', 1306.0999539659297),
 ('ETSU', 1307.7035714020321),
 ('Incarnate Word', 1315.2278894323636),
 ('Idaho St.', 1326.9412271058495),
 ('UC Davis', 1331.6904265171165),
 ('Norfolk St.', 1335.5960932238554),
 ('Florida A&M', 1336.1859120785418),
 ('Presbyterian', 1340.6955021094373),
 ('South Dakota', 1341.9550154484

In [163]:
ivy_team_names = ['Yale','Harvard','Princeton','Cornell','Brown','Columbia','Dartmouth','Penn']
ivy_elo_ratings_dict = {}
for tpl in sorted_elo_rankings[::-1]:
    if tpl[0] in ivy_team_names:
        ivy_elo_ratings_dict[tpl[0]] = tpl[1]

In [164]:
ivy_elo_ratings_dict

{'Brown': 1483.962590862064,
 'Columbia': 1248.9326673570238,
 'Cornell': 1255.826315531858,
 'Dartmouth': 1638.3716743940108,
 'Harvard': 1781.550447813401,
 'Penn': 1505.6293240739337,
 'Princeton': 1721.5801144483294,
 'Yale': 1461.7473543971141}